### Whale Activity vs. Price Movement, combining Coingecko and Dune Sim API data

#### Installing/ Importing Libraries

In [ ]:
! pip install requests pandas numpy matplotlib seaborn joblib python-dotenv dune-client
! pip freeze > requirements.txt

In [18]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
from dotenv import load_dotenv
from datetime import datetime, timedelta
import joblib
from dune_client.client import DuneClient

#### Fetching Data from CoinGecko

In [19]:
coingecko_url = 'https://pro-api.coingecko.com/api/v3' # CoinGecko API endpoint

In [20]:
API_KEY = os.getenv('GECKO_API')  # Fetching the CoinGecko API key from environment variables

if not API_KEY:
    raise ValueError('GECKO_API environment variable not set.')

In [21]:
headers = {
    'accept': 'application/json',
    'x-cg-pro-api-key': API_KEY
}

In [22]:
# Check API Server Status

response = requests.get(coingecko_url + '/ping', headers=headers)
response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx status code)
data = response.json()  # Parsing the JSON response

print(f'API Response:\n{json.dumps(data, indent=4)}')  # Pretty print the JSON data with an indentation of 4 spaces

API Response:
{
    "gecko_says": "(V3) To the Moon!"
}


In [23]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [24]:
coin = 'wrapped-bitcoin'

params = {
    'vs_currency': 'usd', # Currency to get the data in
    'days': '60', # Number of days to get the data for
    'interval': 'daily' # Interval for the data (daily, hourly, etc.)
}

response = requests.get(coingecko_url + f'/coins/{coin}/market_chart',  params=params, headers=headers) 
data = response.json() # Parsing the JSON response

print(f'API Response:\n{json.dumps(data, indent=4)}')

API Response:
{
    "prices": [
        [
            1749340800000,
            105676.8796711252
        ],
        [
            1749427200000,
            105489.28577420597
        ],
        [
            1749513600000,
            110054.259484131
        ],
        [
            1749600000000,
            110266.38569729491
        ],
        [
            1749686400000,
            108580.94170578528
        ],
        [
            1749772800000,
            105877.79118626309
        ],
        [
            1749859200000,
            105966.131119303
        ],
        [
            1749945600000,
            105418.57093229842
        ],
        [
            1750032000000,
            105441.49480725655
        ],
        [
            1750118400000,
            106994.38016298799
        ],
        [
            1750204800000,
            104716.57616853649
        ],
        [
            1750291200000,
            104798.9432337692
        ],
        [
            1750

In [25]:
# Convert result into dataframe 

data = response.json()

# Convert 'prices' to DataFrame
df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

print(df.head())

   timestamp          price
0 2025-06-08  105676.879671
1 2025-06-09  105489.285774
2 2025-06-10  110054.259484
3 2025-06-11  110266.385697
4 2025-06-12  108580.941706


In [26]:
import os
import requests
import pandas as pd
import joblib

class CoinGeckoAPI:
    def __init__(self):
        self.base_url = "https://api.coingecko.com/api/v3"

    def get_price_data(self, coin_id, days=30, interval='daily'):
        """Fetch historical price data and return as DataFrame"""
        url = f"{self.base_url}/coins/{coin_id}/market_chart"
        params = {
            'vs_currency': 'usd',
            'days': days,
            'interval': interval
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            prices = data['prices']
            df = pd.DataFrame(prices, columns=['timestamp', 'price'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            return df
        except Exception as e:
            print(f"Error fetching data: {e}")
            return None

    def save_price_data(self, data, filename):
        """Save DataFrame as a .pkl file"""
        db_path = r"C:/Users/FFFO CASHIER PT/OneDrive/Desktop/Mini Capstone project/database"
        os.makedirs(db_path, exist_ok=True)
        joblib.dump(data, os.path.join(db_path, f"{filename}_prices.pkl"))


In [27]:
cg = CoinGeckoAPI()
df = cg.get_price_data('wrapped-bitcoin', days=90, interval='daily')
if df is not None:
    cg.save_price_data(df, 'wrapped-bitcoin')

#### Fetch data from Dune API

In [28]:
from dotenv import load_dotenv
import os
from dune_client.client import DuneClient

In [29]:
load_dotenv()
api_key = os.getenv("DUNE_API_KEY")
if not api_key:
    raise ValueError("DUNE_API_KEY environment variable not set.")

In [30]:
from dune_client.client import DuneClient

dune = DuneClient(api_key)
query_result = dune.get_latest_result(5589770)

In [33]:
# Access the result rows (list of dictionaries)
rows = query_result.result.rows  # This is my API response data

for row in rows[:10]:  # Preview first 10 rows
    print(row)

{'receiver': '0x9a62db4c17146172c0b88e7e439df169a0f93e0e', 'sender': '0xa3a7b6f88361f48403514059f1f16c8e78d60eec', 'time': '2025-08-06 18:56:11.000 UTC', 'usd_value': 155736.40071057912}
{'receiver': '0xa4b9569bf942c3aad23c0c2d322fe4aff8e1bf30', 'sender': '0xd49a3ff72739e3fe9537645acad3ba3e65f6690d', 'time': '2025-08-06 18:51:35.000 UTC', 'usd_value': 2450903.7932258183}
{'receiver': '0x2a49eae5cca3f050ebec729cf90cc910fadaf7a2', 'sender': '0xfa8c996e158b80d77fbd0082bb437556a65b96e0', 'time': '2025-08-06 18:32:23.000 UTC', 'usd_value': 122574.9476284326}
{'receiver': '0xd49a3ff72739e3fe9537645acad3ba3e65f6690d', 'sender': '0x39c1cc6e689f001567f80b279277f921ce88e6a5', 'time': '2025-08-06 18:29:11.000 UTC', 'usd_value': 2454708.104163917}
{'receiver': '0x51c72848c68a965f66fa7a88855f9f7784502a7f', 'sender': '0xe8f7c89c5efa061e340f2d2f206ec78fd8f7e124', 'time': '2025-08-06 18:28:11.000 UTC', 'usd_value': 136861.58748397118}
{'receiver': '0x51c72848c68a965f66fa7a88855f9f7784502a7f', 'sender'

In [ ]:
# Convert to DataFrame 

import pandas as pd
df = pd.DataFrame(rows)
print(df.head())

                                     receiver  \
0  0x9a62db4c17146172c0b88e7e439df169a0f93e0e   
1  0xa4b9569bf942c3aad23c0c2d322fe4aff8e1bf30   
2  0x2a49eae5cca3f050ebec729cf90cc910fadaf7a2   
3  0xd49a3ff72739e3fe9537645acad3ba3e65f6690d   
4  0x51c72848c68a965f66fa7a88855f9f7784502a7f   

                                       sender                         time  \
0  0xa3a7b6f88361f48403514059f1f16c8e78d60eec  2025-08-06 18:56:11.000 UTC   
1  0xd49a3ff72739e3fe9537645acad3ba3e65f6690d  2025-08-06 18:51:35.000 UTC   
2  0xfa8c996e158b80d77fbd0082bb437556a65b96e0  2025-08-06 18:32:23.000 UTC   
3  0x39c1cc6e689f001567f80b279277f921ce88e6a5  2025-08-06 18:29:11.000 UTC   
4  0xe8f7c89c5efa061e340f2d2f206ec78fd8f7e124  2025-08-06 18:28:11.000 UTC   

      usd_value  
0  1.557364e+05  
1  2.450904e+06  
2  1.225749e+05  
3  2.454708e+06  
4  1.368616e+05  


In [35]:
def fetch_dune_data(query_id, api_key):
    from dune_client.client import DuneClient

    dune = DuneClient(api_key)
    query_result = dune.get_latest_result(query_id)
    rows = query_result.result.rows  # List of dicts
    return rows
# For Dune
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("DUNE_API_KEY")

data = fetch_dune_data(query_id=5589770, api_key=api_key)
import joblib
import os

# Define a save path
save_path = r"C:/Users/FFFO CASHIER PT/OneDrive\Desktop/Mini Capstone project/database"
os.makedirs(save_path, exist_ok=True)

# Save as .pkl
joblib.dump(data, os.path.join(save_path, "dune_data.pkl"))


['C:/Users/FFFO CASHIER PT/OneDrive\\Desktop/Mini Capstone project/database\\dune_data.pkl']